In [1]:
import pandas as pd
import numpy as np

In [2]:
AFS = pd.read_excel('AFS_in_bedrijf_24_04_2018.xlsx', sheetname=None)

In [3]:
S = pd.read_excel('S_24_04_2018.xlsx', sheetname=None)

In [4]:
stations = S['G station']
stations = stations.fillna('Leeg')
#stations = stations[stations['Status'] == 'In bedrijf'] # soms zijn afsluiters nog gekoppeld aan stations buiten bedrijf
stations = stations.rename(columns = {'Id' : 'station_id'})
stations = stations.rename(columns = {'Soort' : 'station_soort'})
stations = stations.rename(columns = {'Status' : 'station_status'})

In [13]:
afsluiters = AFS['G afsluiter']
afsluiters = afsluiters.fillna('Leeg')
#afsluiters = afsluiters[afsluiters['Jaar van eerste aanleg'] == 2017]
#afsluiters = afsluiters[afsluiters['Opgetuigd J/N'] == 'Ja']
afsluiters['G inlaat'] = afsluiters['G inlaat'].replace(0.0, 'Leeg') # een 0.0 station ID wordt als net_afsluiter beschouwd
afsluiters['G uitlaat'] = afsluiters['G uitlaat'].replace(0.0, 'Leeg') # een 0.0 station ID wordt als net_afsluiter beschouwd
afsluiters = afsluiters.rename(columns = {'Id' : 'afsluiter_id'})
afsluiters = afsluiters.rename(columns = {'Soort' : 'afsluiter_soort'})

# voor het UP moet ook nog rekening gehouden worden met of het een plugafsluiter is, voor de data verrijking niet
afsluiters_inlaat = afsluiters[afsluiters['G inlaat'] != 'Leeg']
afsluiters_inlaat = afsluiters_inlaat.rename(columns = {'G inlaat' : 'station_id'})
afsluiters_inlaat = afsluiters_inlaat.drop(['G uitlaat'], axis=1)

# soms is een afsluiter zowel inlaat als uitlaat van een station, als dat zo is worden ze meegenomen bij de inlaatafsluiters
afsluiters_uitlaat = afsluiters[(afsluiters['G uitlaat'] != 'Leeg') & (afsluiters['G inlaat'] == 'Leeg')]
afsluiters_uitlaat = afsluiters_uitlaat.rename(columns = {'G uitlaat' : 'station_id'})
afsluiters_uitlaat = afsluiters_uitlaat.drop(['G inlaat'], axis=1)

station_afsluiters = afsluiters_uitlaat.append(afsluiters_inlaat)


In [38]:
afsluiters[afsluiters['Functie'] == 'Netafsluiter'].groupby('afsluiter_soort').size()

afsluiter_soort
Axiaal                       40
Diafragma                    31
Kogelkraan                 3996
Kogelkraan met reductie      35
Kogelkraan/Las               24
Membraan                     24
O-Ring                       80
Onbekend                   4745
Plugkraan                  1492
Schuif                     7401
Schuif/Klep                 719
Vlinderklep                 492
dtype: int64

In [41]:
afsluiters[(afsluiters['Functie'] == 'Scheidingsafsluiter') &
           (afsluiters['G inlaat'] == 'Leeg') &
           (afsluiters['G uitlaat'] == 'Leeg') &
           (afsluiters['Stand'] == 'Dicht')].shape

(99, 27)

In [10]:
afsluiters[afsluiters['Functie'] == 'Scheidingsafsluiter'].shape

(5022, 27)

In [11]:
merge = pd.merge(station_afsluiters, stations, on='station_id').loc[:, ['afsluiter_id',
                                                                        'Functie',
                                                                        'station_id',
                                                                        'Stationnummer/-naam',
                                                                        'station_soort',
                                                                        'Druk',
                                                                        'station_status']]

In [ ]:
# maak mogelijkheden steeds kleiner door te filteren naar Type, Diameter, Druk en soort

In [10]:
# AS en HAS met gekoppelde afsluiters

scheidingsafsluiters = pd.DataFrame()
scheidingsafsluiters.loc[:,'Huisaansluitset'] = merge[merge['station_soort'] == 'Huisaansluitset'].groupby('Functie').size()
scheidingsafsluiters.loc[:,'Afleverstation'] = merge[merge['station_soort'] == 'Afleverstation'].groupby('Functie').size()
scheidingsafsluiters

,Huisaansluitset,Afleverstation
Functie,,
Brandafsluiter,1,1
Netafsluiter,44,14
Onbekend,2,3
Scheidingsafsluiter,2117,92
Scheidingsafsluiter inlaat,2050,1815
Scheidingsafsluiter uitlaat,59,75


In [12]:
#merge[(merge['station_soort'] == 'Huisaansluitset') & (merge['Functie'] == 'Scheidingsafsluiter uitlaat')]

In [11]:
# DS met hoge druk hoort inlaat te zijn en met lage druk uitlaat

DS_afsluiters = pd.DataFrame()
DS_afsluiters.loc[:,'LD'] = merge[(merge['station_soort'] == 'Districtstation') &
                                  (merge['Druk'].str.contains('mbar'))].groupby('Functie').size()
DS_afsluiters.loc[:,'HD'] = merge[(merge['station_soort'] == 'Districtstation') &
                                  (merge['Druk'].str.contains(' bar'))].groupby('Functie').size()
DS_afsluiters.fillna(0).astype(int)

,LD,HD
Functie,,
Brandafsluiter,3,2
Netafsluiter,34,32
Onbekend,2,0
Scheidingsafsluiter,974,859
Scheidingsafsluiter inlaat,17,1261
Scheidingsafsluiter uitlaat,1379,8


In [40]:
merge[(merge['station_soort'] == 'Districtstation') & (merge['Functie'] == 'Scheidingsafsluiter uitlaat') &
      (merge['Druk'].isin(['1000', '2000', '3000', '4000', '8000']))]

,afsluiter_id,Functie,station_id,Stationnummer/-naam,station_soort,Druk,station_status
241,1.2551e+06,Scheidingsafsluiter uitlaat,1.48862e+08,1,Districtstation,8000,In bedrijf
242,1.48947e+08,Scheidingsafsluiter uitlaat,1.48862e+08,1,Districtstation,8000,In bedrijf
287,1.26166e+06,Scheidingsafsluiter uitlaat,1.36243e+06,1333,Districtstation,1000,In bedrijf
1253,2.43821e+07,Scheidingsafsluiter uitlaat,2.61909e+07,203031060,Districtstation,8000,In bedrijf
1333,2.43863e+07,Scheidingsafsluiter uitlaat,2.62249e+07,203033216,Districtstation,8000,In bedrijf
1439,2.43792e+07,Scheidingsafsluiter uitlaat,2.61615e+07,203079051,Districtstation,8000,In bedrijf
3434,1.53603e+08,Scheidingsafsluiter uitlaat,1.52882e+08,DSH88,Districtstation,8000,In bedrijf
3727,1.57884e+08,Scheidingsafsluiter uitlaat,2.22282e+08,2811.DS.018,Districtstation,3000,In bedrijf


In [12]:
# zet de druk om naar mbar om zo hogere en lagere drukken gemakkelijker te vergelijken
merge['Druk'] = merge['Druk'].replace('\s', '', regex= True)
merge['Druk'] = merge['Druk'].replace('mbar', '', regex= True)
merge['Druk'] = merge['Druk'].replace('bar', '000', regex= True)
groep = merge.groupby(['station_id'])['Functie', 'Druk', 'afsluiter_id'].agg(lambda x: list(x))

In [13]:
[groep.iloc[x] for x in range(len(groep)) if len(groep.iloc[x].Functie) == 2][0].Druk

['30', '1000']

In [14]:
# bij dezelfde OS is de hoge druk de inlaat en de lage druk de uitlaat
merge[(merge['station_soort'] == 'Overslagstation')].groupby('Functie').size()

merge[(merge['station_soort'] == 'Overslagstation')].groupby(['station_id','Druk'])['Functie'].apply(list)

station_id   Druk
1365019.0    1000                        [Scheidingsafsluiter uitlaat]
             8000                         [Scheidingsafsluiter inlaat]
1365082.0    1000                        [Scheidingsafsluiter uitlaat]
             8000                         [Scheidingsafsluiter inlaat]
1365235.0    1000    [Scheidingsafsluiter uitlaat, Scheidingsafslui...
             30                          [Scheidingsafsluiter uitlaat]
             8000    [Scheidingsafsluiter inlaat, Scheidingsafsluit...
1365320.0    1000                        [Scheidingsafsluiter uitlaat]
             8000                         [Scheidingsafsluiter inlaat]
1365663.0    1000                        [Scheidingsafsluiter uitlaat]
             8000                         [Scheidingsafsluiter inlaat]
1366122.0    1000                        [Scheidingsafsluiter uitlaat]
             8000                         [Scheidingsafsluiter inlaat]
1366213.0    1000                        [Scheidingsafsluit

In [ ]:
#from pandas import ExcelWriter
#from pandas import ExcelFile
#writer = ExcelWriter('Panda_export.xlsx')
#merge_net.to_excel(writer,'Sheet1',index=False)
#writer.save()

In [ ]:
# percentage stations met gekoppelde afsluiters # dit is lager vanwege alle buiten bedrijf stations die meetellen nu
aantal_stations = stations.shape[0]
aantal_stations_met_afsluiter = afsluiters[(afsluiters['G inlaat'].isin(stations['station_id'].tolist())) |
                               (afsluiters['G uitlaat'].isin(stations['station_id'].tolist()))].shape[0]
percentage_stations_met_afsluiters = 100 * aantal_stations_met_afsluiter / stations.shape[0]

round(percentage_stations_met_afsluiters, 2)

In [ ]:
merge[(merge['station_status'] == 'In bedrijf') & (merge['station_soort'] == 'Ontvangstation') & (merge['Druk'] == '4 bar')].afsluiter_id.astype(int)

In [ ]:
# in bedrijf zijnde stations met gekoppelde afsluiters

station_aantal = stations[stations['station_status'] == 'In bedrijf'].groupby('station_soort').nunique()['station_id']
gekoppelde_aantal = merge[merge['station_status'] == 'In bedrijf'].groupby('station_soort').nunique()['station_id']

koppeling = pd.DataFrame()
koppeling.loc[:, 'Aantal stations'] = station_aantal
koppeling.loc[:, 'Aantal gekoppelde stations'] = gekoppelde_aantal
koppeling.loc[:, 'Percentage'] = round(100 * koppeling.loc[:, 'Aantal gekoppelde stations'] /
                                       koppeling.loc[:, 'Aantal stations'], 1)
koppeling

In [ ]:
AFS_dataframe = pd.DataFrame()

# lagedrukinstallaties zijn stations, maar tellen niet als dusdanig mee
LS_merge = merge[merge['station_soort'] == 'Lagedrukinstallatie']
LS_HD = LS_merge[LS_merge['Druk'].str.contains(' bar')].shape[0]
LS_LD = LS_merge[LS_merge['Druk'].str.contains('mbar')].shape[0]

# Ontvangstation zijn stations, maar tellen niet als dusdanig mee
GOS_merge = merge[merge['station_soort'] == 'Ontvangstation']
GOS_HD = GOS_merge[GOS_merge['Druk'].str.contains(' bar')].shape[0]
GOS_LD = GOS_merge[GOS_merge['Druk'].str.contains('mbar')].shape[0]

LD_net_extra = merge_net['Druk'].str.contains('mbar').sum() # dit zijn de afsluiters gekoppeld met stations die niet in gebruik zijn
HD_net_extra = merge_net['Druk'].str.contains(' bar').sum()

AFS_dataframe.loc['HD', 'net'] = net_afsluiters[net_afsluiters['Druk'].str.contains(' bar')].count()[0] + LS_HD + GOS_HD + HD_net_extra
AFS_dataframe.loc['LD', 'net'] = net_afsluiters[net_afsluiters['Druk'].str.contains('mbar')].count()[0] + LS_LD + GOS_LD + LD_net_extra

DS_merge = merge[merge['station_soort'] == 'Districtstation']
AFS_dataframe.loc['HD', 'DS'] = DS_merge[DS_merge['Druk'].str.contains(' bar')].shape[0]
AFS_dataframe.loc['LD', 'DS'] = DS_merge[DS_merge['Druk'].str.contains('mbar')].shape[0]

OS_merge = merge[merge['station_soort'] == 'Overslagstation']
AFS_dataframe.loc['HD', 'OS'] = OS_merge[OS_merge['Druk'].str.contains(' bar')].shape[0]
AFS_dataframe.loc['LD', 'OS'] = OS_merge[OS_merge['Druk'].str.contains('mbar')].shape[0]

AS_merge = merge[merge['station_soort'] == 'Afleverstation']
AFS_dataframe.loc['HD', 'AS'] = AS_merge[AS_merge['Druk'].str.contains(' bar')].shape[0]
AFS_dataframe.loc['LD', 'AS'] = AS_merge[AS_merge['Druk'].str.contains('mbar')].shape[0]

HAS_merge = merge[merge['station_soort'] == 'Huisaansluitset']
AFS_dataframe.loc['HD', 'HAS'] = HAS_merge[HAS_merge['Druk'].str.contains(' bar')].shape[0]
AFS_dataframe.loc['LD', 'HAS'] = HAS_merge[HAS_merge['Druk'].str.contains('mbar')].shape[0]

AFS_dataframe.loc['HD', 'plug'] = plug_afsluiters[plug_afsluiters['Druk'].str.contains(' bar')].shape[0]
AFS_dataframe.loc['LD', 'plug'] = plug_afsluiters[plug_afsluiters['Druk'].str.contains('mbar')].shape[0]

AFS_dataframe['totaal'] = AFS_dataframe.sum(axis=1)

In [ ]:
AFS_dataframe

In [ ]:
afsluiters.shape[0], AFS_dataframe.totaal.sum()